In [2]:
import pandas as pd
import numpy as np

In [8]:
eve_df = pd.read_csv('/Users/amir/Desktop/Projects/Fakenews/Fack-News-Checker/Dataset/evaluation.csv', sep=';')
test_df = pd.read_csv('/Users/amir/Desktop/Projects/Fakenews/Fack-News-Checker/Dataset/test (1).csv', sep=';')
train_df = pd.read_csv('/Users/amir/Desktop/Projects/Fakenews/Fack-News-Checker/Dataset/train (2).csv', sep=';')

In [4]:
train_df.head()


,Unnamed: 0,title,text,label
0,0,"Sanders back in U.S. Senate, blasts 'coloniali...",WASHINGTON (Reuters) - Democratic U.S. preside...,1
1,1,Kremlin: Syria peoples' congress being 'active...,MOSCOW (Reuters) - A proposal to convene a con...,1
2,2,Oregon Cop Convicted Of Shattering Biker’s Co...,"In a baffling fit of rage, an Oregon State Pol...",0
3,3,Twitter Erupts With Glee Over #CruzSexScandal...,The last thing any politician running for the ...,0
4,4,MUST WATCH VIDEO: Obama Tries To Trash Trump B...,This is too good to miss! Mr. Teleprompter did...,0


In [5]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8117 entries, 0 to 8116
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  8117 non-null   int64 
 1   title       8117 non-null   object
 2   text        8117 non-null   object
 3   label       8117 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 253.8+ KB


In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report


In [9]:
tfidf = TfidfVectorizer(
    max_features=5000,  # Limit features to top 5000 most frequent words
    min_df=5,          # Ignore terms that appear in less than 5 documents
    max_df=0.95,       # Ignore terms that appear in more than 95% of documents
    stop_words='english'  # Remove common English stop words
)
# 2. Transform the data
# Fit and transform training data
X_train_tfidf = tfidf.fit_transform(train_df['title'])  # Only use title as per your requirement
X_eval_tfidf = tfidf.transform(eve_df['title'])
X_test_tfidf = tfidf.transform(test_df['title'])



In [10]:
lr_model = LogisticRegression(max_iter=1000, C=1.0 , class_weight= 'balanced')

In [11]:
# Train the model
lr_model.fit(X_train_tfidf, train_df['label'])

# 4. Make predictions on evaluation set
eval_predictions = lr_model.predict(X_eval_tfidf)



In [12]:
# 5. Print performance metrics
print("Evaluation Set Performance:")
print(classification_report(eve_df['label'], eval_predictions))

Evaluation Set Performance:
              precision    recall  f1-score   support

           0       0.92      0.90      0.91      3803
           1       0.91      0.93      0.92      4314

    accuracy                           0.92      8117
   macro avg       0.92      0.91      0.92      8117
weighted avg       0.92      0.92      0.92      8117



In [14]:
def predict_news(title):
      # Transform the title using our trained TF-IDF
    title_tfidf = tfidf.transform([title])
    # Get prediction (0 for fake, 1 for real)
    prediction = lr_model.predict(title_tfidf)[0]
    # Get probability scores
    probability = lr_model.predict_proba(title_tfidf)[0]

    result = {
        'prediction':'Real News' if prediction == 1 else 'Fake News',
        'confidence': f'{max(probability)*100:.2f}%'
    }
    return result

In [15]:
# 2. Save the model and vectorizer for future use
import joblib

# Save model and vectorizer
joblib.dump(lr_model, 'fake_news_model.joblib')
joblib.dump(tfidf, 'tfidf_vectorizer.joblib')


['tfidf_vectorizer.joblib']

In [16]:
example_titles = [
    "Scientists discover new species in Amazon rainforest",
    "SHOCKING: Aliens spotted in White House!!!"
]

In [17]:
for title in example_titles:
    result = predict_news(title)
    print(f"\nTitle: {title}")
    print(f"Prediction: {result['prediction']}")
    print(f"Confidence: {result['confidence']}")
    


Title: Scientists discover new species in Amazon rainforest
Prediction: Fake News
Confidence: 50.72%

Title: SHOCKING: Aliens spotted in White House!!!
Prediction: Fake News
Confidence: 78.87%
